<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/marco-canas/taca/blob/main/ref/geron/chap_2/5_prepare_for_algorithms/2_data_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
  <td>
    <a target="_blank" href="https://kaggle.com/kernels/welcome?src=https://github.com/marco-canas/taca/blob/main/ref/geron/chap_2/5_prepare_for_algorithms/2_data_cleaning.ipynb"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" /></a>
  </td>
</table>

# 2 Data Cleaning   
Página 101  

## [Video de apoyo]() 

La mayoría de los algoritmos de aprendizaje automático no pueden funcionar con los atributos que falten, así que creemos algunas funciones para solucionar el problema cuando falten datos. 

In [ ]:
import numpy as np 
import pandas as pd 

In [ ]:
url = 'https://raw.githubusercontent.com/marco-canas/taca/main/datasets/housing/housing.csv'
housing = pd.read_csv(url)

In [ ]:
housing.head(4) 

In [ ]:
housing['income_cat'] = pd.cut(housing['median_income'], 
                              bins = [0,1.5,3.0,4.5,6.0, np.inf], 
                              labels = [1,2,3,4,5])

In [ ]:
housing.head() 

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit 

In [ ]:
barajado = StratifiedShuffleSplit(n_splits = 1, test_size = 0.2, random_state = 513) 

In [ ]:
n = 0
for train_indices, test_indices in barajado.split(housing, housing['income_cat']):
    n+=1
    strat_train_set = housing.loc[train_indices]
    strat_test_set = housing.loc[test_indices] 
    
n     

In [ ]:
housing = strat_train_set.drop(['income_cat', 'median_house_value'], axis = 1).copy() 

In [ ]:
housing_label = strat_train_set['median_house_value'] 

In [ ]:
housing.info() 

Vemos anteriormente que el atributo `total_bedrooms` tiene algunos valores faltantes, así que solucionemos esto. 

Tienes tres opciones:  

1. Deshazte de los distritos correspondientes.
2. Deshazte de todo el atributo.
3. Establezca los valores en algún valor (cero, la media, la mediana, etc.).

Puede lograr esto fácilmente usando los métodos 
* `dropna()`, 
* `drop()` y 
* `fillna()`  

de DataFrame:

In [ ]:
housing.dropna(subset=["total_bedrooms"])     # option 1
housing.drop("total_bedrooms", axis=1)        # option 2
median = housing["total_bedrooms"].median()   # option 3
housing["total_bedrooms"].fillna(median, inplace=True)

Si elige la opción 3, debe calcular el valor de la mediana en el conjunto de entrenamiento y usarlo para completar los valores faltantes en el conjunto de entrenamiento.

No olvide guardar el valor de la mediana que ha calculado. 

Lo necesitará más adelante para reemplazar los valores faltantes en el conjunto de prueba cuando desee evaluar su sistema, y también una vez que el sistema entre en funcionamiento para reemplazar los valores faltantes en nuevos datos.

Scikit-Learn proporciona una clase útil para ocuparse de los valores faltantes: `SimpleImputer`.

A continuación se explica cómo utilizarlo.

Primero, debe crear una instancia de SimpleImputer, especificando que desea reemplazar los valores faltantes de cada atributo con la mediana de ese atributo:

In [ ]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy="median")

Dado que la mediana solo se puede calcular en atributos numéricos, debe crear una copia de los datos sin el atributo de texto `ocean_proximity`:

In [ ]:
housing_num = housing.drop("ocean_proximity", axis=1)

Ahora puede ajustar la instancia del imputador a los datos de entrenamiento usando el método `fit()`:

In [ ]:
imputer.fit(housing_num)


El imputador simplemente calculó la mediana de cada atributo y almacenó el resultado en su variable de instancia `statistics_`.

Solo el atributo `total_bedrooms` tenía valores faltantes, pero no podemos estar seguros de que no faltarán valores en los datos nuevos después de que el sistema entre en funcionamiento, por lo que es más seguro aplicar el imputador a todos los atributos numéricos:

In [ ]:
imputer.statistics_

In [ ]:
housing_num.median().values

Ahora puede usar este imputador "entrenado" para transformar el conjunto de entrenamiento reemplazando los valores faltantes con las medianas aprendidas:

In [ ]:
X = imputer.transform(housing_num)

El resultado es una matriz NumPy simple que contiene las características transformadas.

Si desea volver a colocarlo en un `DataFrame` de pandas, es simple:

In [ ]:
housing_tr = pd.DataFrame(X, columns = housing_num.columns, index = housing_num.index)


## DISEÑO SCIKIT-LEARN

La API de Scikit-Learn está muy bien diseñada. Estos son los principales principios de diseño:

### Consistencia  

Todos los objetos comparten una interfaz simple y consistente:

#### Estimadores

Cualquier objeto que pueda estimar algunos parámetros basándose en un conjunto de datos se denomina estimador (por ejemplo, un imputador es un estimador).

La estimación en sí se realiza mediante el método `fit()` y solo toma un conjunto de datos como parámetro (o dos para los algoritmos de aprendizaje supervisado; el segundo conjunto de datos contiene las etiquetas).
 

Cualquier otro parámetro necesario para guiar el proceso de estimación se considera un hiperparámetro (como la estrategia de un imputador) y debe establecerse como una variable de instancia (generalmente a través de un parámetro de constructor).

### Transformadores

Algunos estimadores (como un imputador) también pueden transformar un conjunto de datos; estos se llaman transformadores. Una vez más, la API es simple: la transformación se realiza mediante el método `transform()` con el conjunto de datos a transformar como parámetro. Devuelve el conjunto de datos transformado.

Esta transformación generalmente se basa en los aprendidos parámetros, como es el caso de un imputador.

Todos los transformadores también tienen un método de conveniencia llamado fit_transform () que es equivalente a
llamando a `fit()` y luego `transform()` (pero a veces `fit_transform()` está optimizado y se ejecuta mucho más rápido).

### Predictores

Finalmente, algunos estimadores, dado un conjunto de datos, son capaces de hacer predicciones; se llaman predictores. Por ejemplo, el modelo LinearRegression del capítulo anterior fue un predictor:

dado el PIB per cápita de un país, predijo la satisfacción con la vida.

Un predictor tiene un método predict () que toma un conjunto de datos de nuevos
instancias y devuelve un conjunto de datos de predicciones correspondientes. También tiene un método score () que mide la calidad de las predicciones, dado un conjunto de pruebas (y las etiquetas correspondientes, en el caso de algoritmos de aprendizaje supervisado).

### Inspección

Todos los hiperparámetros del estimador son accesibles directamente a través de variables de instancia públicas (por ejemplo, imputer.strategy), y todos los parámetros aprendidos del estimador son accesibles a través de variables de instancia públicas con un sufijo de subrayado (por ejemplo, imputer.statistics_).

### No proliferación de clases

Los conjuntos de datos se representan como matrices NumPy o matrices dispersas SciPy, en lugar de clases caseras. Los hiperparámetros son solo cadenas o números normales de Python.

### Composición

Los bloques de construcción existentes se reutilizan tanto como sea posible. Por ejemplo, es fácil crear un estimador Pipeline a partir de una secuencia arbitraria de transformadores seguida de un estimador final, como veremos.

### Defaults sensibles

Scikit-Learn proporciona valores predeterminados razonables para la mayoría de los parámetros, lo que facilita la creación rápida de un sistema de trabajo básico.